In [ ]:
import pandas as pd
import numpy as np
import altair as alt

Part 1: data preparation

1. how many columns are there in data?

2. how many rows are there in the data dictionary table? (Hint, it should be equal to the number of columns in data)

3. set the columns names in data according to their Field Name in the data dictionary table. Show the first five rows and first five columns from data. You will need to parse the data dictionary table HTML file. Please do not set the column names manually!

4. Show the last five rows and last five columns from data.

5. What are the columns containing credit score in any casing? Return a list with the same order of columns in data
 
6. Calculate the average of these columns, round with hround return the results in a series

7. As you see, only Borrower Credit Score at Origination and Co-Borrower Credit Score at Origination contain data. We want to create a new column Combined Credit Score with the average of these two columns. The average should be a value even when one of the columns is null. If one of the columns is null, the average should simply be the other non-null column. If both of them are not null the average is the matehmatical average (i.e., (first + second) / 2). Show the first five and last five rows and the columns Borrower Credit Score at Origination, Co-Borrower Credit Score at Origination, and Combined Credit Score

8. Calculate the average Combined Credit Score per Property State, round the results with hround. Return the results in a series, sort by index.

9. We want to calculate the min and max of Combined Credit Score per Property State. Then calculate the range (max - min) in each state. Return the results in a data frame where the state is the index.

10. What is the state with highest range.

11. We want to calculate the frist quartile (q1) and the third quartile (q3) of Combined Credit Score per Property State. Then calculate the interquartile range (q3 - q1) in each state. Return the results in a data frame where the state is the index.

12. What is the state with highest iqr?

In this homework, we have data from Fannie Mae Single-Family Loan Performance Data

The data file south_east_092012.csv is given to you. This file contain records from the Fannie Mae Single-Family Loan Performance Data for the South Eastern states of AL, GA, FL, MS, NC, SC, TN for loans originating on 09/2012.

The data file does not have a header. The header is provided in the HTML page data dictionary

In [ ]:
data = pd.read_csv('south_east_092012.csv', header=None, delimiter='|')
url = 'https://www.cs.mcgill.ca/~hsafad/files/crt-file-layout-and-glossary.html'
data.head()

In [ ]:
test_1 = len(data.columns)

In [ ]:
tables = pd.read_html(url,encoding='windows-1252',header=0)
data_dictionary = tables[0]
test_2 = len(data_dictionary['Field Position'])

In [ ]:
data.columns = data_dictionary['Field Name'].values
test_3 = data.iloc[:5,:5]

In [ ]:
test_4 = data.iloc[-5:,-5:]

In [ ]:
test_5 = data.columns[data.columns.str.lower().str.contains('credit score')].to_list()

In [ ]:
test_6 = hround(data[['Borrower Credit Score at Origination',
 'Co-Borrower Credit Score at Origination',
 'Borrower Credit Score At Issuance',
 'Co-Borrower Credit Score At Issuance',
 'Borrower Credit Score Current',
 'Co-Borrower Credit Score Current']].mean())

In [ ]:
data['Combined Credit Score'] = hround((data['Borrower Credit Score at Origination'].fillna(data['Co-Borrower Credit Score at Origination'])  + 
                           data['Co-Borrower Credit Score at Origination'].fillna(data['Borrower Credit Score at Origination']))/2)
first_five = data[['Borrower Credit Score at Origination','Co-Borrower Credit Score at Origination','Combined Credit Score']].iloc[:5]
last_five =  data[['Borrower Credit Score at Origination','Co-Borrower Credit Score at Origination','Combined Credit Score']].iloc[-5:]
test_7 = pd.concat([first_five,last_five])

In [ ]:
test_8 = hround(data[['Property State','Combined Credit Score']].groupby('Property State').mean().iloc[:,0])

In [ ]:
def abs_difference(x,y):
    return(abs(x-y))

In [ ]:
credit_aggs = data[['Property State', 'Combined Credit Score']].groupby('Property State').aggregate(
    min=('Combined Credit Score', 'min'),
    max=('Combined Credit Score', 'max')
)
credit_aggs['range'] = abs_difference(credit_aggs['min'],credit_aggs['max'])
test_9 = credit_aggs

In [ ]:
test_10 = test_9['range'].idxmax()

In [ ]:
def q1(x):
    return(x.quantile(0.25))

def q3(x):
    return(x.quantile(0.75))

def iqr(q3,q1):
    return(q3-q1)

In [ ]:
credit_quantiles = data[['Combined Credit Score','Property State']].groupby('Property State').aggregate(
    q1 = ('Combined Credit Score',q1),
    q3 = ('Combined Credit Score',q3)
)
credit_quantiles['iqr'] = iqr(credit_quantiles['q3'],credit_quantiles['q1'])
test_11 = credit_quantiles

In [ ]:
test_12 = credit_quantiles['iqr'].idxmax()